# Convolution network


``` tf.nn.conv2d()```  :Convolution network operation in TensorFlow.

``` tf.nn.max_pool()```:Max pooling 

## Check shape of tensor by calling 'print'. 

```
l1a = tf.nn.relu(~~~)
$print (l1a)
```





In [2]:
import time
import tensorflow as tf
import numpy as np

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    
    # Conv2d
    # l1a shape=(?, 28, 28, 32)
    # padding='SAME' means output data's dimension is same as input image's. 
    l1a = tf.nn.relu( 
            tf.nn.conv2d(
                X, w,
                strides=[1,1,1,1],
                padding='SAME') 
        )
    
    # Max pooling 
    # l1 shape=(?, 14, 14, 32)
    l1 = tf.nn.max_pool(
            l1a, ksize=[1, 2, 2, 1],              
            strides=[1, 2, 2, 1], 
            padding='SAME'
        )
    
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    # Conv2d
    # l2a shape=(?, 14, 14, 64)
    l2a = tf.nn.relu( 
            tf.nn.conv2d(
                l1, w2,
                strides=[1,1,1,1],
                padding='SAME') 
        )

    # l2 shape=(?, 7, 7, 64)
    l2 = tf.nn.max_pool(
        l2a, ksize=[1, 2, 2, 1],              
        strides=[1, 2, 2, 1], 
        padding='SAME'
    )
    
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    # l3a shape=(?, 7, 7, 128)
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     
                        strides=[1, 1, 1, 1], padding='SAME'))
    
    # l3 shape=(?, 4, 4, 128)
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],             
                        strides=[1, 2, 2, 1], padding='SAME')
    
    # reshape to (?, 2048)
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])   
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    # Fully connected neural network
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx
    
    
    
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img

X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])


w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)


# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        training_batch = zip(range(0, len(trX), batch_size),
                             range(batch_size, len(trX), batch_size))
        
        start_time = time.time()
        print("Begin training...")
        # Training...
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})
        training_time = time.time() 
        elapsed_time = training_time - start_time
        print(elapsed_time, "secs passed.")
        
        # Get A Test Batch
        test_indices = np.arange(len(teX)) 
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        # Testing...
        print("Begin testing...")
        print('step:',i,' Accuracy:', np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))
        elapsed_time = time.time() - training_time
        print(elapsed_time, "secs passed.")



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Begin training...
135.2799210548401 secs passed.
Begin testing...
step: 0  Accuracy: 0.96484375
0.2369699478149414 secs passed.
Begin training...
133.61649107933044 secs passed.
Begin testing...
step: 1  Accuracy: 0.98046875
0.22284388542175293 secs passed.
Begin training...
129.91897201538086 secs passed.
Begin testing...
step: 2  Accuracy: 0.9765625
0.21730709075927734 secs passed.
Begin training...
129.28189396858215 secs passed.
Begin testing...
step: 3  Accuracy: 0.98828125
0.21044206619262695 secs passed.
Begin training...
123.06760716438293 secs passed.
Begin testing...
step: 4  Accuracy: 0.98046875
0.2186